In [2]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.sandbox.regression.gmm import GMM
from datetime import datetime

In [3]:
data = pd.read_csv('bus1234.csv')
data['day'] = 1
data['day'] = pd.to_datetime(data[['year','month','day']])
data['miles'] =data['miles']/1e6

In [17]:
class test(GMM):
    
    def __init__(self,*args, shift=50, gap=1, nmoms=2, **kwds, ):
        # set appropriate counts for moment conditions and parameters
        kwds.setdefault('k_moms', nmoms)
        kwds.setdefault('k_params',2)
        self.shift = shift
        self.gap = gap
        super(test, self).__init__(*args, **kwds)
        
        
    def momcond(self, params):
        
        params = np.clip(params,-100, -.01 )
        
        data = self.instrument.copy()
        #hacky way to ensure shock is negative
        data['shock'] =  data['replace']*data['shock'] #- 1*(data['replace']==0)*data['shock']
        
        data['util'] = params[0]*data['miles']*(1-data['replace']) + params[1]*data['replace']
        data['util_shock'] = data['shock'] + data['util']
        u_avg = data['util'].mean()
        
        moms = []
        for i in [0,1]:
            u_lag = data.groupby('id')['util'].shift(self.shift)
            u_lag_avg = ( u_lag[data['replace']==i] ).mean()
            moms.append( (data['util_shock']*u_lag - u_avg*u_lag_avg) )
            
        moms = pd.DataFrame(np.array(moms).transpose())
        moms = moms.dropna()
        return moms
    
data['shock'] = sm.OLS(data['replace'], sm.add_constant( data['miles'] ) ).fit().resid
#data['shock'] = sm.Logit(data['replace'], sm.add_constant( data['miles']/1e6 ) ).fit().resid_response

data = data.sort_values(['id','day'])  
model = test(data['replace'],data['miles'],data[['id','miles','replace','day','shock']], shift=10)
result = model.fit(np.array([-.5,-10]),optim_method='nm')
result.summary()

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 31
         Function evaluations: 93
Optimization terminated successfully.
         Current function value: 0.007609
         Iterations: 12
         Function evaluations: 47
Optimization terminated successfully.
         Current function value: 0.007609
         Iterations: 12
         Function evaluations: 47
Optimization terminated successfully.
         Current function value: 0.007609
         Iterations: 12
         Function evaluations: 47


LinAlgError: Singular matrix